In [ ]:
from langchain import PromptTemplate

In [ ]:
template = """You are a naming bot for music bands. What is a good name for a band that plays {genre}?"""

In [ ]:
prompt = PromptTemplate.from_template(template)
prompt.format(genre="jazz metal lofi")

In [ ]:
import inspect 
def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)


In [ ]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

In [ ]:
class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):

    @validator("input_variables")
    def validate_input_variables(cls,v):
        if len(v)!=1 or "function_name" not in v:
            raise ValueError("function_name must be the only input variable")
        return v
    
    def format(self, **kwargs) -> str:
        source_code = get_source_code(kwargs["function_name"])

        prompt=f"""   
        Given the function name and source code, generate an English language explanation of the function.
        Function Name: {kwargs["function_name"].__name__}
        Source Code:
        {source_code}
        Explanation:
        """
        return prompt
    
    def prompt_type(self):
        return "function-explainer"

        

In [ ]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

prompt = fn_explainer.format(function_name=get_source_code)

In [ ]:
print(prompt)

In [ ]:
#chat models have a system message 

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
prompt=PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)
system_message_prompt_2 = SystemMessagePromptTemplate(prompt=prompt)

assert system_message_prompt == system_message_prompt_2

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()

In [ ]:
import sys
sys.prefix

In [ ]:
sys.base_prefix

In [ ]:
from pprint import pp
pp(sys.path)

In [ ]:
from feast import FeatureStore
from langchain.prompts import PromptTemplate, StringPromptTemplate

feast_repo_path = "../../../../../my_feature_repo/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

template = """Given certain stats, write a note relaying those stats
If they have a x stat above y, give compliment, otherwise provide feedback

Here are the stats:

X stat: {x_rate}
Y stat: {y_rate}
Z stat: {z_rate}

Your response:"""

prompt = PromptTemplate.from_template(template)


In [ ]:
class FeastPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                "driver_hourly_stats:conv_rate",
                "driver_hourly_stats:acc_rate",
                "driver_hourly_stats:avg_daily_trips",  
            ],
            entity_rows=[{"driver_id": driver_id}],
        ).to_dict()
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)


In [ ]:
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])
print(prompt_template.format(driver_id=1001))

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

#create formatter for few shot examples

"""Configure formatter for few shot to string as a
PromptTemplate object"""

example_prompt = PromptTemplate(input_variables=)

prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
 )

print(prompt.format(input="who was the father of Mary Ball Washington?"))

In [ ]:
"""Time to use ExampleSelector
- Uses SemanticSimilarityExampleSelector class
- Selects few shot examples based on similarity to the input.
- Uses an embeddings model to compute similarity to the input
  and few shot examples as well as vector store to perform
  nearest neighbor search.
"""

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings,
    Chroma,
    k=1
)


question = "some question here?"
selected_examples = example_selector.select_examples({"question": question})
print(f"examples similar to input: {question}")

for example in selected_examples:
    for k,v in example.items():
        print(f"{k}:{v}")

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}"
    input_variables=["input"]
)

In [ ]:
#Few shot examples for chat models
#alternating human and AI messages
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import(
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key=os.getenv("OPENAI_API_KEY")

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates english to pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_human, example_ai, human_message_prompt]
)
chain = LLMChain(llm=chat, prompt=chat_prompt)
# get a chat completion from the formatted messages
chain.run("I love programming.")

In [ ]:
chain.run("I love typing sentences such as: the quick brown fox jumps over the lazy dog")

In [ ]:
"""Types of MEssafePromptTemplate
-AIMessagePT, SysMPT, HumanMPT
If chat model supports taking chat message with arbitrary role
we have ChatMessagePromptTemplate
"""
from langchain.prompts import ChatMessagePromptTemplate
prompt = "May the {subject} be with you"

cm_prompt = ChatMessagePromptTemplate.from_template(role="jedi", template = prompt).format(subject="force")
cm_prompt

In [ ]:
#We also get MessagesPlaceholder which gives full control
#over what messages to be rendered during formatting

from langchain.prompts import MessagesPlaceholder

human_prompt = "Summarize our conversation so far in {word_count} words"

hm_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([MessagesPlaceholder(variable_name="conversation"), hm_template])

human_message = HumanMessage(content="What is the best way to learn programming?")

ai_message=AIMessage(content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
""")
                     
chat_prompt.format_prompt(conversation=[human_message, ai_message], word_count="10").to_messages()




In [ ]:
#partial prompt templates - basically you pass only one of the input variables if another one is coming later on in the chain. For example: there is a prompt template that requires two variables, one you get early on in the chain but the other one you only get later on in the chain. Thus you would partial the prompt with the earlier obtained variable and pass along the partialed template. Another use case is with functions. 

#for example:

from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")


prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}", 
    input_variables=["adjective", "date"]
);
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

In [ ]:
#You can also just initialize the prompt with the partialed variables

prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}", 
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime}
);
print(prompt.format(adjective="funny"))

In [ ]:
#now we come to composition - composing multiple prompts together. For this we shall use PipelinePrompt.

from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

full_template = """{introduction}

{example}

{start}"""

full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}"""

introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""

example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q:{input}
A:"""

start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]

pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)


In [ ]:
pipeline_prompt.input_variables

In [ ]:
print(pipeline_prompt.format(
    person="Elon Musk",
    example_q="What are your favorite Tesla cars in order most to least?",
    example_a="That's simple: S, E, X, Y",
    input="Who is your least favorite person on Twitter?"
))

In [ ]:
# Now we are doing serialization - storing prompts not as python code but as files. Following design principles are applicable:

# Both JSON and YAML are supported - two popular human readable on disk formats

# Specifying everything on one file is supported, or storing different components (templates, examples, etc) in different files and then referencing them. 

"""
cat simple_template.txt

    Tell me a {adjective} joke about {content}.

cat simple_prompt_with_template_file.json

    {
        "_type": "prompt",
        "input_variables": ["adjective", "content"],
        "template_path": "simple_template.txt"
    }
"""
from langchain.prompts import load_prompt
prompt = load_prompt("simple_prompt_with_template_file.json")
print(prompt.format(adjective="funny", content="chickens"))

# Output: Tell me a funny joke about chickens.


In [ ]:
# More examples using few shot for json, yaml, text file

# custom example selector

# ExampleSelector must implement two methods: add_example takes in example and adds it into the ExampleSelector, select_examples takes in input variables and returns a list of examples to use in the few shot prompt.


from langchain.prompts.example_selector.base import BaseExampleSelector

from typing import Dict, List
import numpy as np


class CustomExampleSelector(BaseExampleSelector):
     def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples
    
     def add_example(self, example: Dict[str, str]) -> None:
        """Add new example to store for a key."""
        self.examples.append(example)

     def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Select which examples to use based on the inputs."""
        return np.random.choice(self.examples, size=2, replace=False)

